## 3. Библиотека requests

Перед началом работы библиотеку requests потребуется установить. Например, в Jupyter Notebook это делается с помощью такой команды:

!pip install requests 

Как только библиотека установлена, импортируем её и отправим наш первый запрос к ресурсу Курсы валют ЦБ РФ в XML и JSON. Используем метод get() из библиотеки requests, передав ему соответствующий URL —  https://www.cbr-xml-daily.ru/daily_json.js:

In [19]:
import requests # Импортируем библиотеку requests
url = 'https://www.cbr-xml-daily.ru/daily_json.js' # Определяем значение URL страницы для запроса
response = requests.get(url) # Делаем GET-запрос к ресурсу и результат ответа сохраняем в переменной response

In [20]:
# Проверим ответ сервера — содержимое переменной response:
print(response) # Выводим значение response на экран как объект

<Response [200]>


Код ответа в виде числовой переменной можно получить с помощью метода status_code:

In [21]:
print(response.status_code) # Выводим числовое значение response на экран

200


Напишите строку кода, при выполнении которой будет сделан GET-запрос к ресурсу https://www.cbr-xml-daily.ru/daily.xml, а результат ответа будет сохранён в переменной response:

In [22]:
# response = requests.get('https://www.cbr-xml-daily.ru/daily.xml')

Текст ответа хранится в атрибуте text. Выведем значение атрибута на экран и посмотрим на его содержимое:

In [23]:
print(response.text) # Выводим содержимое атрибута text переменной response на экран

{
    "Date": "2024-06-12T11:30:00+03:00",
    "PreviousDate": "2024-06-11T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2024\/06\/11\/daily_json.js",
    "Timestamp": "2024-06-12T20:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 58.7719,
            "Previous": 59.3771
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 52.3655,
            "Previous": 52.3496
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 113.2085,
            "Previous": 113.2365
 

Для того чтобы удобно было работать с полученной информацией, нам необходимо преобразовать строку в словарь. В объект ответа Response  из библиотеки requests уже встроен метод json() .

Импортируем функцию pprint(), применим к полученному ответу метод json() и выведем полученный результат на экран:

In [24]:
from pprint import pprint # Импортируем функцию pprint()
import json # Импортируем модуль json
currencies = response.json() # Применяем метод json()
pprint(currencies) # Выводим результат на экран

{'Date': '2024-06-12T11:30:00+03:00',
 'PreviousDate': '2024-06-11T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2024/06/11/daily_json.js',
 'Timestamp': '2024-06-12T20:00:00+03:00',
 'Valute': {'AED': {'CharCode': 'AED',
                    'ID': 'R01230',
                    'Name': 'Дирхам ОАЭ',
                    'Nominal': 1,
                    'NumCode': '784',
                    'Previous': 24.2326,
                    'Value': 24.24},
            'AMD': {'CharCode': 'AMD',
                    'ID': 'R01060',
                    'Name': 'Армянских драмов',
                    'Nominal': 100,
                    'NumCode': '051',
                    'Previous': 22.9639,
                    'Value': 22.9685},
            'AUD': {'CharCode': 'AUD',
                    'ID': 'R01010',
                    'Name': 'Австралийский доллар',
                    'Nominal': 1,
                    'NumCode': '036',
                    'Previous': 59.3771,
              

Теперь данные находятся в словаре и можно легко получать необходимые значения.

Например, по ключу Valute мы можем обратиться к вложенному словарю, который содержит информацию о мировых валютах. Выведем на экран, например, информацию о евро (EUR):

In [25]:
pprint(currencies['Valute']['EUR']) # Выводим на экран информацию о валюте евро

{'CharCode': 'EUR',
 'ID': 'R01239',
 'Name': 'Евро',
 'Nominal': 1,
 'NumCode': '978',
 'Previous': 95.6367,
 'Value': 95.7391}


In [26]:
print(currencies['Valute']['CZK']['Name'])

Чешских крон


## 4. Парсинг сайтов

Для примера рассмотрим страницу, содержащую статью с информацией о присуждении Нобелевской премии по экономике в 2021 году, и попробуем извлечь из неё заголовок статьи, опубликованной на странице, дату публикации, а также текст статьи.

Получить содержимое страницы в большинстве случаев несложно, гораздо труднее извлечь из HTML-кода нужную информацию.

Вы можете ознакомиться с информацией о HTML в справочнике, перейдя по ссылке. http://htmlbook.ru/html

#### ПОЛУЧАЕМ СОДЕРЖИМОЕ ВЕБ-СТРАНИЦЫ

Получим HTML-код интересующей нас страницы.

Для этого отправим GET-запрос с помощью библиотеки requests и метода get() и посмотрим на текст ответа на наш запрос (как мы помним, он содержится в атрибуте text):

In [27]:
import requests # Импортируем библиотеку requests
url = 'https://nplus1.ru/news/2021/10/11/econobel2021' # Определяем адрес страницы
response = requests.get(url)  # Выполняем GET-запрос
print(response.text)  # Выводим содержимое атрибута text

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#" lang="ru">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>
    <link rel="preload" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" as="font" type="font/woff2" crossorigin />
    <link rel="preload" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" as="font" type="font/woff2" crossorigin />
  <link href="/front-build/css/main.css?id=9b496fbb252428c03791da99722528ab" rel="stylesheet">
  <link href="/front-build/css/app.css?id=446d52496c231791fc49bcdc58828382" rel="stylesheet">
  

  <link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png">
  <link rel="icon" type="image/png" sizes="32x32" href="/favicon-32x32.png">
  <link rel="icon" type="image/png" sizes="16x16" href="/favicon-16x16.png">
  <link rel="

Для решения таких задач в Python существует специальная библиотека BeautifulSoup, о работе с которой мы поговорим в следующем юните.

## 5. Библиотека BeautifulSoup

BeautifulSoup не является частью стандартной библиотеки, поэтому для начала её нужно установить. Например, в Jupyter Notebook это делается с помощью такой команды: pip install beautifulsoup4 

In [28]:
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup

Теперь мы можем извлекать данные из любой веб-страницы.

Ранее мы уже получили содержимое страницы с помощью GET-запроса и сохранили информацию в переменной response , теперь создадим объект BeautifulSoup с именем page, указывая в качестве параметра html.parser.

Для примера получим информацию o title (с англ. заголовок) — это строка, которая отображается на вкладке браузера:

In [29]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
url = 'https://nplus1.ru/news/2021/10/11/econobel2021' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
print(page.title) # Получаем тег title, отображающийся на вкладке браузера
print(page.title.text) # Выводим текст из полученного тега, который содержится в атрибуте text

<title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>
Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей


Если при запросе к сайту, а затем при его разборе с помощью BeautifulSoup в тексте страницы не находится нужный тег, попробуйте вывести на печать пару тысяч символов текста страницы. Если там обнаружится нечто похожее на капчу, возможно, сайт посчитал вас роботом и отказывается выдавать содержимое. Чтобы получить его, попробуйте «притвориться» браузером при запросе из скрипта:

In [30]:
requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

<Response [200]>

### ИЗВЛЕКАЕМ ЗАГОЛОВОК СТАТЬИ

Выполним поставленную ранее задачу: получить информацию о странице и извлечь заголовок статьи, опубликованной на этой странице, дату публикации, а также текст статьи.

Предположим, что мы знаем, что в HTML-коде рассматриваемой нами страницы заголовок статьи заключён в тег (заголовок первого уровня).

Тогда мы можем получить его текст с помощью метода find() (с англ. найти) объекта BeautifulSoup, передав ему название интересующего нас тега:

In [31]:
# Применяем метод find() к объекту и выводим результат на экран
print(page.find('h1').text) 


            Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей
          


Проще всего это сделать с помощью так называемого инструмента разработчика, который есть во всех современных браузерах. Покажем, как открыть данный инструмент на примере использования браузера Google Chrome.

Устанавливаем курсор на элементе страницы (заголовок статьи), информацию о котором хотим получить, нажимаем на правую клавишу мыши и в выпадающем списке выбираем пункт «Просмотреть код элемента» или «Исследовать» в зависимости от браузера.

Напишите функцию wiki_header, которая по адресу страницы возвращает заголовок первого уровня для статей на Wikipedia.

Функция wiki_header принимает один аргумент - url.

In [32]:
# импортируем библиотеки
import requests
from bs4 import BeautifulSoup

def wiki_header(url):
    # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    response = requests.get(url)
    # Создаём объект BeautifulSoup, указывая html-парсер
    soup = BeautifulSoup(response.content, 'html.parser')
    # находим заголовок и записываем его в переменную
    title = soup.find(id="firstHeading")
    return title.string

### НЕУНИКАЛЬНЫЕ ТЕГИ: ИЗВЛЕКАЕМ ТЕКСТ И ДАТУ ПУБЛИКАЦИИ СТАТЬИ

Теперь получим сам текст статьи. Как вы уже знаете, первым делом необходимо определить, в какой тег он заключён. Применим, как и ранее, инструмент разработчика.

Посмотрим на нашу страницу, используя инструмент разработчика, ещё раз. Можем заметить, что у искомого текста есть свой класс — n1_material text-18 :

Передадим название класса в метод find() с помощью аргумента class_ и получим текст статьи:

In [33]:
print(page.find('div', class_='n1_material text-18').text) 
# Выводим содержимое атрибута text тега div класса n1_material text-18

Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад в эмпирические исследования экономики рынка труда, а также Джошуа Энгристу (Joshua Angrist) и Гвидо Имбенсу (Guido Imbens) за их вклад в методологию анализа причинно-следственных связей. Прямая трансляция церемонии объявления лауреатов шла на официальном сайте Нобелевской премии.


В данном случае происходит поиск точного строкового значения class атрибута, т. е. выполнение строк кода:

Аналогично получим информации о теге, который содержит дату написания статьи, отображаемую в левом верхнем углу страницы.

Теперь получим данные из него с помощью уже известного метода find(), передав название нужного тега:

In [34]:
# Выводим на экран содержимое атрибута text тега a с классом "relative"
print(page.find('a', class_= "relative").text)


11.10.21



О поиске по классу можно узнать подробнее в Beautiful Soup Documentation. https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-by-css-class

### СБОР НЕСКОЛЬКИХ ЭЛЕМЕНТОВ: СОБИРАЕМ ВСЕ ССЫЛКИ НА СТРАНИЦЕ

Рассмотрим ещё один сценарий: вы хотите собрать сразу несколько элементов со страницы. Например, представьте, что вы хотите получить названия всех языков программирования, упомянутых на странице в Wikipedia в статье про языки программирования.

Можно заметить, что все названия языков программирования на этой странице связаны ссылками c соответствующими статьями о них. Таким образом, нам необходимо собрать все ссылки на странице. Для ссылок в HTML предусмотрен тег . Попробуем использовать find():

In [35]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages' # Задаём адрес ресурса
response = requests.get(url) # Делаем GET-запрос к ресурсу
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup
print(page.find('a')) # Ищем ссылку по тегу <a> и выводим её на экран

<a class="mw-jump-link" href="#bodyContent">Jump to content</a>


Мы получили только одну ссылку, хотя на странице их явно больше.

Это происходит, потому что метод find() возвращает только первый подходящий элемент. Если требуется получить больше элементов, необходимо воспользоваться методом find_all() (с англ. найти все):

In [36]:
links = page.find_all('a') 
# Ищем все ссылки на странице и сохраняем в переменной links в виде списка
print(len(links)) 
# Выводим количество найденных ссылок

954


Итак, на момент создания этих учебных материалов на странице содержалось 928 ссылок. Посмотрим на некоторые из них:

In [37]:
print([link.text for link in links[500:510]]) 
# Выводим ссылки с 500 по 509 включительно

['MAD/I', 'Magik', 'Magma', 'Maple', 'MAPPER', 'MARK-IV', 'Mary', 'MATLAB', 'MASM Microsoft Assembly x86', 'MATH-MATIC']


Не все ссылки соответствуют названиям языков программирования — страница содержит также «служебные» ссылки, такие, например, как Jump to navigation (с англ. Перейти к навигации) или Alphabetical (с англ. По алфавиту):

✍ В заключение заметим, что BeautifulSoup — достаточно мощная библиотека. Мы рассмотрели её базовые возможности, но их полный список гораздо шире. С ним можно ознакомиться в [официальной документации](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

## 6. Работа с API

### КЛЮЧ АВТОРИЗАЦИИ

Для того чтобы начать работать с API, обычно необходимо получить сервисный ключ авторизации — токен.

Зайдите на [страницу](https://vk.com/editapp?act=create), чтобы создать приложение (вы должны быть авторизованы ВКонтакте). Дайте приложению любое название и в разделе Платформа поставьте отметку выбора напротив значения Встраиваемое приложение:

### ПЕРВЫЕ ЗАПРОСЫ К API

Перейдите по следующей ниже ссылке в браузере, подставив вместо слова TOKEN ваш персональный сервисный ключ доступа (токен), полученный на предыдущем шаге:

https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=TOKEN

https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=61f3aaf261f3aaf261f3aaf29f62eb35f1661f361f3aaf20799c729af5891b5a26d3cfa

{"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false}]}

Итак, мы сделали GET-запрос к API ВКонтакте, который состоит из следующих элементов:

- https://api.vk.com/method — домен и URL запроса API; обычно не меняется;
- users.get — название метода, который отдаёт определённый отчёт, в нашем случае это метод для получения информации о пользователе;
- user_id и v — параметры запроса: идентификатор пользователя, о котором хотим получить информацию (в нашем примере мы запрашиваем информацию о первом пользователе), и номер версии API;
- token — токен, который выдаётся только пользователям, имеющим право просматривать определённые данные, например показания счётчиков Яндекс.Метрики вашего проекта; на все остальные запросы без корректного токена система отвечает отказом.

Если мы обратимся к документации метода users.get, то увидим, что в ней описано множество других параметров, которые можно получить о пользователе (дата рождения, пол, родной город и другие) — словом, всё то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал).

Добавим к запросу дату рождения и пол (согласно документации, эти параметры надо перечислять в поле fields):

https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=TOKEN

### ЗАПРОС К API ИЗ КОДА

Продолжаем пользоваться всё той же библиотекой requests.

In [38]:
import requests # Импортируем модуль requests
token = '61f3aaf261f3aaf261f3aaf29f62eb35f1661f361f3aaf20799c729af5891b5a26d3cfa' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/users.get' # Указываем адрес страницы к которой делаем запрос
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'} # Перечисляем параметры нашего запроса в словаре params
response = requests.get(url, params=params) # Отправляем запрос
print(response.text) # Выводим текст ответа на экран

{"response":[{"id":1,"bdate":"10.10.1984","sex":2,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false}]}


Мы получили строку в JSON-формате, которую можно преобразовать в словарь с помощью метода json(), после чего можно с лёгкостью обращаться к различным полям.

Словари нагляднее выводить с помощью функции pprint(), которую мы уже использовали ранее:

In [39]:
from pprint import pprint # Импортируем функцию pprint()
pprint(response.json()) # Выводим содержимое словаря, содержащего ответ, на экран

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


Как вы видите, по ключу response мы можем получить список, в котором хранятся словари, содержащие информацию о запрошенных нами пользователях. Мы запросили информацию лишь об одном из них, поэтому список содержит только один элемент. Извлечём его:

In [40]:
user = response.json()['response'][0] # Извлекаем из словаря по ключу response информацию о первом пользователе
print(user['bdate']) # Выводим дату рождения первого пользователя на экран
print(user['first_name']) # выведет на экран имя пользователя

10.10.1984
Павел


Метод users.get() позволяет запрашивать информацию о множестве (до 1 000) пользователей одновременно. Для этого нужно использовать параметр user_ids и передавать id через запятую в строковом формате. Например, чтобы получить информацию о пользователях с id=1, id=2, id=3, необходимо передать значение параметра user_ids='1,2,3'.

Попробуем это сделать:

In [41]:
ids = ",".join(map(str, range(1, 4))) 
# Формируем строку, содержащую информацию о поле id первых трёх пользователей
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bdate', 'access_token': token, 'lang': 'ru'} # Формируем строку параметров
pprint(requests.get(url, params=params).json()) 
# Посылаем запрос, полученный ответ в формате JSON-строки преобразуем в словарь и выводим на экран его содержимое, используя функцию pprint()

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'bdate': '14.2',
               'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'can_access_closed': True,
               'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'is_closed': False,
               'last_name': ''}]}


Используя API, определите долю женщин (sex=1) среди пользователей с id от 1 до 500. Иногда будут попадаться пользователи, у которых пол не указан (sex=0), — таких пользователей не нужно учитывать в общем числе.

В ответе укажите число, округлив до двух знаков после точки-разделителя, например, 0.55.

In [43]:
import requests
token = '61f3aaf261f3aaf261f3aaf29f62eb35f1661f361f3aaf20799c729af5891b5a26d3cfa'
url = 'https://api.vk.com/method/users.get'
ids = ",".join(map(str, range(1, 501)))
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'}
response = requests.get(url, params=params).json()['response']
men=women=0
for elem in response:
    # Проверяем, есть ли ключ 'sex' в словаре
    if 'sex' in elem:
        if elem['sex'] == 2:
            men+=1
        elif elem['sex'] == 1:
            women+=1
    else:
        continue
print(round(women/(men+women),2))


0.53


### СБОР ИНФОРМАЦИИ ИЗ ГРУПП

Теперь мы научимся считать произвольные метрики групп, собирая данные из API и работая с двумя ограничениями, которые свойственны практически всем системам:

- ограничение на количество вызовов в единицу времени;
- ограничение на количество выгружаемых строк за один запрос.

Обратимся к [документации](https://vk.com/dev/groups), чтобы узнать, какие методы нам доступны для групп, — для получения списка пользователей группы доступен метод groups.getMembers.

Согласно документации, обязательным параметром данного метода является group_id — идентификатор, или короткое имя, группы. В нашем случае это vk: https://vk.com/vk. Протестируем, как работает метод в самом простом случае, — получим id участников группы:

In [44]:
import requests 
# Импортируем модуль requests
token = '61f3aaf261f3aaf261f3aaf29f62eb35f1661f361f3aaf20799c729af5891b5a26d3cfa' 
# Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' 
# Указываем адрес обращения
params = {'group_id': 'vk', 'v': 5.95, 'access_token': token} 
# Формируем строку параметров
response = requests.get(url, params = params) 
# Посылаем запрос
data = response.json() 
# Ответ сохраняем в переменной data в формате словаря
print(data) 
# Выводим содержимое переменной data на экран (отображён фрагмент)

{'response': {'count': 13547052, 'items': [6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 296, 345, 421, 431, 450, 467, 485, 510, 550, 619, 640, 690, 702, 721, 804, 809, 832, 834, 847, 900, 905, 907, 914, 930, 943, 952, 958, 966, 976, 997, 1000, 1018, 1023, 1032, 1033, 1038, 1039, 1059, 1097, 1131, 1139, 1140, 1159, 1174, 1185, 1188, 1301, 1333, 1334, 1351, 1381, 1386, 1388, 1406, 1411, 1418, 1432, 1494, 1503, 1531, 1550, 1568, 1586, 1590, 1593, 1598, 1610, 1615, 1632, 1634, 1650, 1679, 1690, 1697, 1698, 1699, 1700, 1721, 1740, 1754, 1796, 1814, 1820, 1829, 1834, 1839, 1840, 1843, 1858, 1863, 1869, 1887, 1889, 1917, 1943, 1947, 1955, 1969, 2019, 2028, 2050, 2051, 2052, 2059, 2077, 2103, 2136, 2150, 2195, 2201, 2230, 2236, 2273, 2281, 2296, 2298, 2376, 2389, 2395, 2403, 2412, 2436, 2456, 2466, 2470, 2484, 2513, 2515, 2539, 2571, 2576, 2592, 2601, 2622, 2644, 2654, 2692, 2706, 2745, 2755, 2767, 2787, 2797, 2827, 2858, 2909, 2919, 2922, 2932, 2953, 2962, 2982, 3001, 3006, 3041, 3130, 31

По ключу count мы можем получить общее число участников группы, а список по ключу items хранит их id. Посмотрим на него поближе:

In [45]:
print(len(data['response']['items'])) 
# Выводим на экран количество элементов словаря

1000


Мы видим, что всего пользователей в группе больше 11 миллионов, а получили мы только первую тысячу пользователей группы. По информации, указанной в документации о параметре count, это максимум, который может отдать API за один раз.

Для получения следующей тысячи пользователей можно воспользоваться параметром offset (с англ. смещение), который передвинет начало отсчёта. Для выгрузки всех пользователей группы будем в цикле выгружать по 1000 пользователей (count будет всегда равен 1000), увеличивая смещение offset на величину count.

Для тренировки напишем цикл выгрузки первых 20 пользователей со значением count=5. Иными словами, мы будем выгружать по пять пользователей за запрос до тех пор, пока не получим информацию о 20 пользователях.

Давайте выведем на экран первые 20 пользователей из нашей первой попытки получить информацию о 1000 пользователей, чтобы мы могли сверить результат выгрузки из 20 пользователей:

In [46]:
users_for_checking = data['response']['items'][:20] 
# Загружаем в переменную информацию об id первых 20 пользователей в виде списка
print(users_for_checking) 
# Выводим перечень id первых 20 пользователей

[6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 296, 345, 421, 431, 450, 467, 485, 510, 550]


In [47]:
import requests 
# Импортируем модуль requests
token = '61f3aaf261f3aaf261f3aaf29f62eb35f1661f361f3aaf20799c729af5891b5a26d3cfa' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' 
# Указываем адрес обращения
count = 5 
offset = 0 
user_ids = [] 
max_count = 20 
while offset < max_count: 
    # Будем выгружать по count=5 пользователей, 
    # начиная с того места, где закончили на предыдущей итерации (offset) 
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))   
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    # Увеличиваем смещение на количество строк, которое мы уже выгрузили 
    offset += count 
print(user_ids) 

Выгружаю 5 пользователей с offset = 0
Выгружаю 5 пользователей с offset = 5
Выгружаю 5 пользователей с offset = 10
Выгружаю 5 пользователей с offset = 15
[6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 296, 345, 421, 431, 450, 467, 485, 510, 550]


Сравним списки, полученные двумя способами:

In [48]:
print(user_ids == users_for_checking) 

True


Так как результат сравнения — True, списки идентичны. Значит, второй способ работает корректно. Теперь мы можем получить данные обо всех пользователях, выставив count = 1000 и max_count = data['response']['count'].

### ОГРАНИЧЕНИЕ ПО ЧАСТОТЕ ЗАПРОСОВ

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, так как периодичность отправки запросов будет искусственно замедлена.

Воспользуемся библиотекой time и методом sleep, с помощью которого мы можем добавить паузу, например в 0.5 секунд, после каждого запроса:

In [49]:
import requests 
# Импортируем модуль requests
import time 
# Импортируем модуль time
token = '61f3aaf261f3aaf261f3aaf29f62eb35f1661f361f3aaf20799c729af5891b5a26d3cfa' 
# Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' 
# Указываем адрес страницы, к которой делаем запрос
count = 1000 
offset = 0  
user_ids = []  
while offset < 5000: 
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    offset += count 
    print('Ожидаю 0.5 секунды...') 
    time.sleep(0.5) 
print('Цикл завершен, offset =',offset) 

Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Цикл завершен, offset = 5000


### ЛАЙКИ, РЕПОСТЫ И КОММЕНТАРИИ

Через API новостной ленты ВКонтакте мы можем получить информацию о взаимодействии с сообщениями в ленте.

Для примера продолжим работать с группой https://vk.com/vk и рассмотрим последние 100 сообщений в новостной ленте.

Для получения информации о сообщениях на стене в API ВКонтакте предусмотрен метод wall.get. Применим его:

In [50]:
import requests 
# Импортируем модуль requests
from pprint import pprint 
# Импортируем функцию pprint()
token = '61f3aaf261f3aaf261f3aaf29f62eb35f1661f361f3aaf20799c729af5891b5a26d3cfa' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/wall.get' 
# Указываем адрес страницы, к которой делаем запрос
params = {'domain': 'vk', 'filter': 'owner', 'count': 1000, 'offset': 0, 'access_token': token, 'v': 5.95} 
response = requests.get(url, params = params) 
pprint(response.json()) 

{'response': {'count': 732,
              'items': [{'attachments': [{'photo': {'access_key': '8e87ecc4188ce8e516',
                                                    'album_id': -7,
                                                    'date': 1718102941,
                                                    'id': 457350616,
                                                    'owner_id': -22822305,
                                                    'post_id': 1490421,
                                                    'sizes': [{'height': 130,
                                                               'type': 'm',
                                                               'url': 'https://sun6-23.userapi.com/impg/yuOQCCGPTwhGNCQA1tb6PVN99pOxQDLM_lD54w/3mVy-QAmx3Y.jpg?size=130x130&quality=96&sign=13e2e8e269ae76c50a0db8d04bf41cb3&c_uniq_tag=tbh8_y7LR-Rhe7rs_oupB0nNvDjSDSJrXSDLYl-WT9s&type=album',
                                                               'width': 130},
       

In [51]:
# Посмотрим на количество результатов:
len(response.json()['response']['items'])
## 100

100

In [52]:
# Посмотрим на информацию об отдельном сообщении:
response.json()['response']['items'][0] 

{'inner_type': 'wall_wallpost',
 'comments': {'count': 149},
 'marked_as_ads': 0,
 'hash': 'Uxml7EoYJ-Ub_sMTMg',
 'type': 'post',
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1718102941,
    'id': 457350616,
    'owner_id': -22822305,
    'access_key': '8e87ecc4188ce8e516',
    'post_id': 1490421,
    'sizes': [{'height': 130,
      'type': 'm',
      'width': 130,
      'url': 'https://sun6-23.userapi.com/impg/yuOQCCGPTwhGNCQA1tb6PVN99pOxQDLM_lD54w/3mVy-QAmx3Y.jpg?size=130x130&quality=96&sign=13e2e8e269ae76c50a0db8d04bf41cb3&c_uniq_tag=tbh8_y7LR-Rhe7rs_oupB0nNvDjSDSJrXSDLYl-WT9s&type=album'},
     {'height': 130,
      'type': 'o',
      'width': 130,
      'url': 'https://sun6-23.userapi.com/impg/yuOQCCGPTwhGNCQA1tb6PVN99pOxQDLM_lD54w/3mVy-QAmx3Y.jpg?size=130x130&quality=96&sign=13e2e8e269ae76c50a0db8d04bf41cb3&c_uniq_tag=tbh8_y7LR-Rhe7rs_oupB0nNvDjSDSJrXSDLYl-WT9s&type=album'},
     {'height': 200,
      'type': 'p',
      'width': 200,
      'url': 'h

В полях comments, likes и reposts содержится статистика по взаимодействию с сообщением пользователей (на момент получения информации) — число комментариев, лайков и репостов.

Давайте соберём итоговую статистику для последних десяти непустых сообщений в словарь stats. В качестве ключа будем использовать начало сообщения (если начало сообщения пустое, то информацию о таком сообщении проигнорируем), в качестве значения — список с тремя интересующими нас метриками и временем публикации (комментарии, лайки, репосты, дата публикации):

In [53]:
stats = {} 
count_post = 0 # Счётчик «непустых» сообщений
for record in response.json()['response']['items'][:]:
    title = record['text'][:30] 
    if title: 
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date']] 
        count_post += 1 
    if count_post < 10: 
        continue 
    else: 
        break 
pprint(stats)

{'В интернете, как и в обычной ж': [347, 770, 118, 1717240211],
 'Загибайте палец, если вы:\n\n1. ': [290, 366, 74, 1716967194],
 'Какие персонажи — такие и их л': [185, 1016, 130, 1716537660],
 'Какой у вас любимый автор клип': [405, 718, 50, 1717760104],
 'Мем смешной, ситуация страшная': [125, 649, 55, 1717151619],
 'Негласное правило ВКонтакте №5': [298, 896, 54, 1717510109],
 'Ой, какое хорошее уведомление!': [149, 699, 50, 1718103606],
 'Правда или действие? Выбирать ': [210, 1229, 122, 1716366595],
 'С добрым уведомлением!': [312, 896, 116, 1716015660],
 'Сегодня — День предпринимателя': [227, 346, 73, 1716721345]}


Мы рассмотрели базовое взаимодействие с пользователями и группами. ВКонтакте предоставляет достаточно широкие возможности в своём API: всё, что можно делать вручную через браузер, доступно и в API.

Если вы размещаете рекламу во ВКонтакте, то можно выгружать всю статистику через [ads API](https://vk.com/dev/ads).

Полный список методов ВКонтакте можно посмотреть в [документации](https://vk.com/dev/methods).

### ДРУГИЕ API

Вы познакомились с интерфейсами прикладного программирования — API (на примере API социальной сети ВКонтакте).

API для разработчиков предоставляют и многие другие платформы. Вот список, пожалуй, самых популярных из них:

Google Maps API https://developers.google.com/maps/

YouTube API https://developers.google.com/youtube/

Twitter API https://dev.twitter.com/overview/documentation

Facebook API https://developers.facebook.com/docs/

Вы также можете воспользоваться интернет-поиском, указав в поисковой строке, например, «курсы валют API» или «прогноз погоды api», — среди первых результатов выдачи чаще всего с лёгкостью можно найти ссылки на необходимый функционал.

* В материалах курса упоминается социальная сеть Facebook/Instagram, принадлежащая Meta Platforms Inc., деятельность которой запрещена на территории РФ в части реализации данных социальных сетей на основании осуществления ею экстремистской деятельности.

## 7. Как настроить регулярную выгрузку данных

Скрипт можно использовать разными способами:

- Если вам нужно выполнить код один раз или выполнять его очень редко (от случая к случаю), вы можете запустить скрипт вручную из командной строки.
- Если вы, например, написали на Python серверную часть веб-сайта и хотите, чтобы ваш код мог в любое время обрабатывать запросы, вы захотите запустить его как фоновый процесс.

А что делать, если вам нужно, чтобы ваш скрипт запускался иногда? Каждую пятницу, 13-го? В день рождения супруга (или супруги)? Или просто каждый час?

В этом случае вам нужен автоматический запуск скриптов, или, как часто его называют программисты, запуск по крону — от английского акронима Cron (англ. Command Run ON) — названия системы для автоматического запуска программ и скриптов на сервере в определённое время.

### КАК НАСТРОИТЬ АВТОМАТИЧЕСКИЙ ЗАПУСК

Во всех операционных системах есть встроенные средства запуска программ по расписанию. Однако можно задать расписание запуска необходимых вам функций на языке Python внутри скрипта.

Исполняемый по расписанию код часто называют задачей (англ. task). Для планирования задач в Python есть несколько библиотек, среди которых — популярный и простой в использовании модуль schedule (c англ. расписание). Он позволяет запускать код как с определённым интервалом, так и в заданное время.

Модуль schedule не входит в стандартную библиотеку Python, поэтому его необходимо установить: pip install schedule 

Для того чтобы у нас появилась возможность использовать модуль в коде, импортируем его:

In [55]:
import schedule # Импортируем модуль schedule

Рассмотрим вариант автоматического запуска простой функции, которая выводит на экран короткое сообщение:

In [56]:
def task(): 
    print('Hello! I am a task!') 
    return 

Предположим, что мы хотим, чтобы функция запускалась через определённые интервалы времени, а именно каждые 15 минут.

Для запуска задачи через определённые интервалы времени в модуле schedule используется метод every(), который получает в качестве единственного аргумента число, указывающее, как часто следует запускать код.

Далее вызывается метод, определяющий единицы измерения промежутков времени, через которые будет выполняться функция. В нашем примере это минуты. Вот как будет выглядеть итоговый код:

In [57]:
schedule.every(15).minutes.do(task)

Every 15 minutes do task() (last run: [never], next run: 2024-06-13 09:58:32)

Если бы мы хотели запускать задачу, например, каждый час, то могли бы написать:

In [58]:
schedule.every(1).hour.do(task) 

Every 1 hour do task() (last run: [never], next run: 2024-06-13 10:43:51)

### ВЫПОЛНЕНИЕ ФУНКЦИИ

После того как мы создали нашу функцию и определились со временем её запуска, мы можем запустить наш менеджер расписания (schedule). Для этого надо создать бесконечный цикл. Да-да, именно так, ведь скрипт должен выполняться постоянно, чтобы постоянно проверять, не пришло ли время снова выполнить задачу.

Внутри цикла мы будем вызывать особый метод run_pending() для объекта schedule, который будет проверять, нет ли задачи, которую пора выполнить.

После вызова метода run_pending() нужно будет сделать небольшую паузу, после которой можно будет снова проверять, не пришло ли время для выполнения какой-либо функции.

Давайте напишем этот код.

Для создания паузы мы будем использовать метод sleep из модуля time, поэтому наш код начнётся с импорта данного модуля:

In [ ]:
import schedule

def task(): 
    print('Hello! I am a task!') 
    return

schedule.every(15).minutes.do(task)

import time 
while True: 
    schedule.run_pending() 
    time.sleep(1)

ПАРСИНГ САЙТОВ

Официальная документация BeautifulSoup https://www.crummy.com/software/BeautifulSoup/bs4/doc/

РАБОТА С API

Описание методов API ВКонтакте https://vk.com/dev/methods

АВТОМАТИЧЕСКИЙ ЗАПУСК СКРИПТОВ

Документация модуля schedule http://schedule.readthedocs.io/

Как работать с планировщиком заданий в ОС Windows https://www.comss.ru/page.php?id=4840

О том, что такое CRON и как его использовать https://tproger.ru/translations/guide-to-cron-jobs/

Советуем вам ознакомиться со статьёй, где собрано 80 самых популярных вопросов по Python, которые задают на собеседованиях.
https://nuancesprog.ru/p/11460/